<a href="https://colab.research.google.com/github/David-Carballo/2023w-project-pose/blob/main/AnglesMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Setup Kaggle
Load kaggle.json file with your Kaggle API Token in Settings>Account>Create New API Token

### Install the Kaggle library

In [3]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# make a directory named ".kaggle"
! mkdir ~/.kaggle
# copy the "kaggle.json" file into this new directory
! cp kaggle.json ~/.kaggle/
# allocate the required permission for this file.
! chmod 600 ~/.kaggle/kaggle.json

### Upload Kaggle Dataset

In [5]:
#Load kaggle.json file with your Kaggle API Token in Settings>Account>Create New API Token
! kaggle datasets download tr1gg3rtrash/yoga-posture-dataset
#unzip dataset
! mkdir data
! unzip yoga-posture-dataset.zip -d data/

 99% 444M/447M [00:13<00:00, 41.8MB/s]
100% 447M/447M [00:13<00:00, 34.1MB/s]
Archive:  yoga-posture-dataset.zip
  inflating: data/Adho Mukha Svanasana/File1.png  
  inflating: data/Adho Mukha Svanasana/File10.png  
  inflating: data/Adho Mukha Svanasana/File11.png  
  inflating: data/Adho Mukha Svanasana/File12.png  
  inflating: data/Adho Mukha Svanasana/File13.png  
  inflating: data/Adho Mukha Svanasana/File14.png  
  inflating: data/Adho Mukha Svanasana/File15.png  
  inflating: data/Adho Mukha Svanasana/File16.png  
  inflating: data/Adho Mukha Svanasana/File17.png  
  inflating: data/Adho Mukha Svanasana/File18.png  
  inflating: data/Adho Mukha Svanasana/File19.png  
  inflating: data/Adho Mukha Svanasana/File2.png  
  inflating: data/Adho Mukha Svanasana/File20.png  
  inflating: data/Adho Mukha Svanasana/File21.png  
  inflating: data/Adho Mukha Svanasana/File22.png  
  inflating: data/Adho Mukha Svanasana/File23.png  
  inflating: data/Adho Mukha Svanasana/File24.png  
  inf

Translate Sanskrit names to English names

In [6]:
import json
import os

poses_dict = {}

with open('data/Poses.json') as f:
    data = json.load(f)
    for pose in data['Poses']:
        poses_dict[pose['sanskrit_name']] = pose['english_name']
    
print(poses_dict)

{'Navasana': 'Boat', 'Ardha Navasana': 'Half-Boat', 'Dhanurasana': 'Bow', 'Setu Bandha Sarvangasana': 'Bridge', 'Baddha Konasana': 'Butterfly', 'Ustrasana': 'Camel', 'Marjaryasana': 'Cat', 'Bitilasana': 'Cow', 'Utkatasana': 'Chair', 'Balasana': "Child's Pose", 'Sivasana': 'Corpse', 'Alanasana': 'Crescent Lunge', 'Bakasana': 'Crow', 'Ardha Pincha Mayurasana': 'Dolphin', 'Adho Mukha Svanasana': 'Downward-Facing Dog', 'Garudasana': 'Eagle', 'Utthita Hasta Padangusthasana': 'Extended Hand to Toe', 'Utthita Parsvakonasana': 'Extended Side Angle', 'Pincha Mayurasana': 'Forearm Stand', 'Uttanasana': 'Standing Forward Bend', 'Ardha Chandrasana': 'Half-Moon', 'Adho Mukha Vrksasana': 'Handstand', 'Anjaneyasana': 'Low Lunge', 'Supta Kapotasana': 'Pigeon', 'Eka Pada Rajakapotasana': 'King Pigeon', 'Phalakasana': 'Plank', 'Halasana': 'Plow', 'Parsvottanasana': 'Pyramid', 'Parsva Virabhadrasana': 'Reverse Warrior', 'Paschimottanasana': 'Seated Forward Bend', 'Padmasana': 'Lotus', 'Ardha Matsyendrasa

# 1. Dataset

## 1.0. Import
Import libraries, OpenPoseModel and set up hyperparameters


In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torch import nn
import torch.nn.functional as F
import torch.optim as optim 
import torchvision.transforms as transforms
import albumentations as A
import numpy as np
from PIL import Image
import os
import random



##1.2 Compute keypoints
Extract keypoints of the images from dataset.

In [2]:
#Load OpenPose Model
!pip install git+https://github.com/prasunroy/openpose-pytorch.git

from openpose.body.estimator import BodyPoseEstimator
from openpose.utils import draw_body_connections, draw_keypoints
import cv2
import math

import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/prasunroy/openpose-pytorch.git to /tmp/pip-req-build-5jx92grd
  Running command git clone --filter=blob:none --quiet https://github.com/prasunroy/openpose-pytorch.git /tmp/pip-req-build-5jx92grd
  Resolved https://github.com/prasunroy/openpose-pytorch.git to commit 8955ef1d220d5dc43dd5e573a49e323393ae5446
  Preparing metadata (setup.py) ... done
  Created wheel for openpose-pytorch: filename=openpose_pytorch-0.1.0-py3-none-any.whl size=10270 sha256=cb74a5273d5bde92104440785eaa63164763ccf6054df91a85d32705b2a12875
  Stored in directory: /tmp/pip-ephem-wheel-cache-xv4bg7hp/wheels/38/fc/56/e5c574e56aedc8f970ef5ad594021411fc25e11375bbe25257
Successfully built openpose-pytorch


###Get keypoints from all images

In [ ]:
data_dir = 'data'
estimator = BodyPoseEstimator(pretrained=True)

index = 0
keypoints = torch.empty(2756,18,3)
#Loop through yoga class names
for i, subdir in enumerate(sorted(os.listdir(data_dir))):
    subdir_path = os.path.join(data_dir, subdir)
    if os.path.isdir(subdir_path):
        # loop through each image of the class
        for file in sorted(os.listdir(subdir_path)):
            file_path = os.path.join(subdir_path, file)
            if file_path.endswith('.jpg') or file_path.endswith('.jpeg') or file_path.endswith('.png'):
                tic = time.perf_counter()
                img = cv2.imread(file_path)
                height, width = img.shape[:2]
                if height > 500 or width > 500:
                    scale_factor = 500.0 / width
                    new_width = int(width * scale_factor)
                    new_height = int(height * scale_factor)
                    # Redimensionar la imagen con el mismo factor de escala en ambas dimensiones
                    img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
                kps = estimator(img)
                toc = time.perf_counter()
                print(f'{index}-{index/2758*100}% -> Img: {file} in {subdir} = {toc - tic:0.4f} seconds')
                if len(kps) == 0:
                    kps = torch.zeros([1,18,3])
                else:
                    kps = torch.from_numpy(kps[0])
                keypoints[index] = kps
                index += 1

0-0.0% -> Img: File1.png in Adho Mukha Svanasana = 4.9994 seconds
1-0.03625815808556925% -> Img: File10.png in Adho Mukha Svanasana = 2.1960 seconds
2-0.0725163161711385% -> Img: File11.png in Adho Mukha Svanasana = 2.8395 seconds
3-0.10877447425670776% -> Img: File12.png in Adho Mukha Svanasana = 2.2625 seconds
4-0.145032632342277% -> Img: File13.png in Adho Mukha Svanasana = 2.3387 seconds
5-0.18129079042784627% -> Img: File14.png in Adho Mukha Svanasana = 2.2032 seconds
6-0.21754894851341552% -> Img: File15.png in Adho Mukha Svanasana = 2.2082 seconds
7-0.25380710659898476% -> Img: File16.png in Adho Mukha Svanasana = 2.3731 seconds
8-0.290065264684554% -> Img: File17.png in Adho Mukha Svanasana = 3.1820 seconds
9-0.3263234227701233% -> Img: File18.png in Adho Mukha Svanasana = 2.0567 seconds
10-0.36258158085569253% -> Img: File19.png in Adho Mukha Svanasana = 1.8445 seconds
11-0.3988397389412618% -> Img: File2.png in Adho Mukha Svanasana = 2.4010 seconds
12-0.43509789702683105% -> 

/usr/local/lib/python3.9/dist-packages/openpose/body/estimator.py:210: RuntimeWarning: invalid value encountered in true_divide
  v = np.divide(v, n)
/usr/local/lib/python3.9/dist-packages/openpose/body/estimator.py:217: RuntimeWarning: divide by zero encountered in double_scalars
  score_with_dist_prior = sum(score_midpoints) / len(score_midpoints) + min(0.5 * image.shape[0] / n - 1, 0)


53-1.9216823785351704% -> Img: File58.png in Adho Mukha Svanasana = 2.1924 seconds
54-1.9579405366207396% -> Img: File59.png in Adho Mukha Svanasana = 1.9276 seconds
55-1.9941986947063088% -> Img: File6.png in Adho Mukha Svanasana = 2.2114 seconds
56-2.030456852791878% -> Img: File60.png in Adho Mukha Svanasana = 2.2284 seconds
57-2.0667150108774472% -> Img: File61.png in Adho Mukha Svanasana = 2.1072 seconds
58-2.102973168963017% -> Img: File62.png in Adho Mukha Svanasana = 3.2106 seconds
59-2.139231327048586% -> Img: File63.png in Adho Mukha Svanasana = 2.0796 seconds
60-2.1754894851341553% -> Img: File64.png in Adho Mukha Svanasana = 2.1796 seconds
61-2.2117476432197245% -> Img: File65.png in Adho Mukha Svanasana = 1.4860 seconds
62-2.2480058013052937% -> Img: File66.png in Adho Mukha Svanasana = 1.7859 seconds
63-2.284263959390863% -> Img: File67.png in Adho Mukha Svanasana = 2.1460 seconds
64-2.320522117476432% -> Img: File68.png in Adho Mukha Svanasana = 3.3880 seconds
65-2.35678

In [ ]:
torch.save(keypoints,'keypoints')

In [269]:
load_keypoints = torch.load('keypoints')

In [314]:
load_keypoints[50]

tensor([[  0.,   0.,   0.],
        [198., 139.,   1.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [180., 139.,   1.],
        [130., 211.,   1.],
        [ 82., 262.,   1.],
        [282.,  47.,   1.],
        [350., 144.,   1.],
        [393., 236.,   1.],
        [294.,  52.,   1.],
        [351., 145.,   1.],
        [405., 241.,   1.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]])

##1.3 Compute Angles

In [371]:
def compute_angle(ax,ay, bx,by, cx,cy):
    dot_product = (bx - ax) * (cx - bx) + (by - ay) * (cy - by)
    mag0 = math.sqrt((bx - ax)**2 + (by - ay)**2)
    mag1 = math.sqrt((cx - bx)**2 + (cy - by)**2)
    angle = math.acos(dot_product / (mag0 * mag1))

    return angle

def get_angles(keypoints):
    n = len(keypoints)
    output = torch.empty(n,12)
    
    i = 0
    for kps in keypoints:
        connections =  [(0,1,2), (1,2,3), (2,3,4), (0,1,5), (1,5,6), (5,6,7), (1,2,8), (2,8,9), (8,9,10), (1,5,11), (5,11,12), (11,12,13)]

        angles = []

        for a,b,c in connections:
            Ax = kps[a][0]
            Ay = kps[a][1]
            Bx = kps[b][0]
            By = kps[b][1]
            Cx = kps[c][0]
            Cy = kps[c][1]
            
            angle_deg = compute_angle(Ax, Ay, Bx, By, Cx, Cy)
            angles.append(angle_deg)
        output[i] = torch.FloatTensor(angles)
        i +=1
    output = torch.nan_to_num(output)
    return output

def get_angles2(kps):
    output = torch.empty(12)
    
    #kps = keypoints[index]
    connections =  [(0,1,2), (1,2,3), (2,3,4), (0,1,5), (1,5,6), (5,6,7), (1,2,8), (2,8,9), (8,9,10), (1,5,11), (5,11,12), (11,12,13)]

    angles = []
    for a,b,c in connections:
        Ax = kps[a][0]
        Ay = kps[a][1]
        Bx = kps[b][0]
        By = kps[b][1]
        Cx = kps[c][0]
        Cy = kps[c][1]
        angle_deg = compute_angle(Ax, Ay, Bx, By, Cx, Cy)
        angles.append(angle_deg)
    output = torch.nan_to_num(torch.FloatTensor(angles))
    return output


In [214]:
angles = get_angles(load_keypoints)

In [392]:
len(angles)


2756

In [370]:
count = 0
n = torch.zeros(12)
for i in range(0,len(angles)):
  if torch.sum(angles[i]) > 0:
      count +=1
  for j in range(0, len(angles[i])):
    if angles[i][j] > 0:
      n[j] +=1
    
print(count)
print(n)

2336
tensor([2316., 2199., 1773., 2316., 2184., 1708., 2292., 1994., 1715., 2268.,
        1973., 1673.])


In [366]:
angles[2000]

tensor([1.4644, 1.6468, 0.0000, 1.5173, 1.0854, 0.2897, 1.5294, 0.8013, 2.6316,
        1.5348, 3.0357, 0.0000])

In [409]:
n = torch.count_nonzero(angles[2000])
print(n)

tensor(10)


In [343]:
a = angles[2000].clone()
def add_noise(angles, alpha):
    new_ang = angles.clone()
    n = random.uniform(-alpha,alpha)*math.pi/180
    
    #new_ang = torch.add(angles, n)
    print(n)
    for i,a in enumerate(angles):
        if angles[i] > 0:
            new_ang[i] = max(0, angles[i] + n)
    return new_ang

print(add_noise(a,30))

-0.13398693040620693
tensor([1.3304, 1.5128, 0.0000, 1.3833, 0.9514, 0.1557, 1.3954, 0.6673, 2.4976,
        1.4008, 2.9018, 0.0000])


In [414]:
images = []
labels = []
for i, subdir in enumerate(sorted(os.listdir('data'))):
          subdir_path = os.path.join('data', subdir)
          if os.path.isdir(subdir_path):
              for file in sorted(os.listdir(subdir_path)):
                  file_path = os.path.join(subdir_path, file)
                  if file_path.endswith('.jpg') or file_path.endswith('.jpeg') or file_path.endswith('.png'):
                      images.append(file_path)
                      labels.append(i)

pose = 0
count = 0
total_angles = 0
for i in range(0, len(labels)):
  n = torch.count_nonzero(angles[i])
  total_angles += n
  count += 1
  if pose != labels[i]:
    pose = labels[i]
    print(f'Class {pose} until position {i} with {count-1} images and {total_angles/count-1} average angles')
    count = 0
    total_angles = 0
print(f'Class {pose+1} in position {i} with {count} images')

Class 1 until position 74 with 74 images and 3.5999999046325684 average angles
Class 2 until position 139 with 64 images and 3.6461539268493652 average angles
Class 3 until position 157 with 17 images and 9.277777671813965 average angles
Class 4 until position 228 with 70 images and 9.873239517211914 average angles
Class 5 until position 287 with 58 images and 9.762711524963379 average angles
Class 6 until position 383 with 95 images and 10.208333015441895 average angles
Class 7 until position 396 with 12 images and 6.846153736114502 average angles
Class 8 until position 450 with 53 images and 4.018518447875977 average angles
Class 9 until position 462 with 11 images and 9.333333015441895 average angles
Class 10 until position 543 with 80 images and 9.333333015441895 average angles
Class 11 until position 627 with 83 images and 8.785714149475098 average angles
Class 12 until position 706 with 78 images and 3.9620251655578613 average angles
Class 13 until position 800 with 93 images and

In [ ]:
# 35, 33, 24, 9, 7, 3

In [219]:
print(angles[0])

def add_noise(angles, alpha):
    new_ang = angles.clone()
    n = random.uniform(-1,1)/math.pi
    #new_ang = torch.add(angles, n)
    for i,a in enumerate(angles):
        if angles[i] > 0:
            new_ang[i] = max(0, angles[i] + n)
    return new_ang

print(add_noise(angles[0], 30))

tensor([3.1416, 0.0000, 0.0000, 1.9839, 0.2810, 0.0789, 2.7937, 3.1416, 0.0000,
        3.0946, 1.6285, 0.0476])
tensor([2.9351, 0.0000, 0.0000, 1.7775, 0.0745, 0.0000, 2.5872, 2.9351, 0.0000,
        2.8882, 1.4220, 0.0000])


##1.4 Dataset

In [382]:
!rm 'data/Poses.json'

class YogaDataset(Dataset):
    def __init__(self, data_dir, transforms=True, keypoints=None):
        self.data_dir = data_dir
        self.images = []
        self.labels = []
        #self.keypoints = keypoints
        self.transforms = transforms
        self.angles = get_angles(keypoints)

        for i, subdir in enumerate(sorted(os.listdir(data_dir))):
            subdir_path = os.path.join(data_dir, subdir)
            if os.path.isdir(subdir_path):
                for file in sorted(os.listdir(subdir_path)):
                    file_path = os.path.join(subdir_path, file)
                    if file_path.endswith('.jpg') or file_path.endswith('.jpeg') or file_path.endswith('.png'):
                        self.images.append(file_path)
                        self.labels.append(i)
        
    def __getitem__(self, index):
        ang = self.angles[index]
        
        #ang = self.angles[index]
        if self.transforms == True:
            if random.random() < 0.75:
                ang = self.add_noise(ang,90)

        #new_ang = get_angles2(ang)
        return ang, self.labels[index]

    def image_transform(self, tensor):
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        resize = transforms.Resize(224)
        #Check if image size ratio is 1:1
        if tensor.shape[1] != tensor.shape[2]:
            #Calculate difference between X and Y axis
            diff = tensor.shape[1] - tensor.shape[2]
        #Check what dimension needs to be padded either the X (dimension 1 of the tensor, right), or the Y (dimension 3, bottom)
            if diff > 0:     
                tensor = F.pad(input=tensor, pad=(0, abs(diff), 0, 0), mode='constant', value=0) #pad(left, right, top, bottom)
            else:
                tensor = F.pad(input=tensor, pad=(0, 0, 0, abs(diff)), mode='constant', value=0)
        tensor = resize(tensor)
        tensor = tensor.reshape(3, 224, 224)
        return tensor

    def add_noise(self,angles, alpha):
        new_ang = angles.clone()
        n = random.uniform(-alpha,alpha) * math.pi / 180
        
        #new_ang = torch.add(angles, n)
        for i,a in enumerate(angles):
            if angles[i] > 0:
                new_ang[i] = max(0, angles[i] + n)
        return new_ang
    
    def __len__(self):
        return len(self.images)

rm: cannot remove 'data/Poses.json': No such file or directory


In [383]:
# Let's define some hyper-parameters
hparams = {
    'batch_size':64,
    'num_epochs':500,
    'test_batch_size':64,
    'hidden_size':32,
    'num_classes':47,
    'num_inputs':12,
    'learning_rate':0.001,
}

# we select to work on GPU if it is available in the machine, otherwise
# will run on CPU
hparams['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

In [384]:
# Test Dataset class
yoga_dataset = YogaDataset('data', True, load_keypoints)

n_train = int(0.8 * len(yoga_dataset))
train_set, test_set = torch.utils.data.random_split(yoga_dataset, [n_train, len(yoga_dataset) - n_train])

n_train = int(0.8 * len(train_set))
train_set, val_set = torch.utils.data.random_split(train_set, [n_train, len(train_set) - n_train])

train_loader = DataLoader(dataset=train_set, batch_size=hparams['batch_size'], shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=hparams['batch_size'], shuffle=False)
test_loader = DataLoader(dataset=test_set, batch_size=hparams['batch_size'], shuffle=False)

# 2. MLP Model

In [385]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.batch = nn.BatchNorm1d(hidden_size)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.drop = nn.Dropout(0.5)
        self.fc2 = nn.Linear(hidden_size, hidden_size*2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size*2, output_size)
        

    def forward(self, x):
        out = self.fc1(x)
        out = self.batch(self.relu1(out))
        out = self.drop(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.drop(out)
        out = self.fc3(out)
        return out

In [386]:
mlp = MLP(hparams['num_inputs'], hparams['hidden_size'] ,hparams['num_classes'])
mlp.to(hparams['device'])

MLP(
  (batch): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=12, out_features=32, bias=True)
  (relu1): ReLU()
  (drop): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=47, bias=True)
)

In [387]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp.parameters(), hparams['learning_rate'])

##Training and Validation

In [388]:
import matplotlib.pyplot as plt

In [389]:
def validate(model, val_loader, criterion, device):
    model.eval()  # set model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data[0].to(device), data[1].to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Calculate statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate accuracy
    accuracy = 100 * correct / total

    # Print validation statistics
    #print('Validation Loss: %.3f, Accuracy: %.3f' % (running_loss / len(val_loader), accuracy))

    return accuracy

In [390]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
  
    model.train()
    train_loss_list = []
    train_acc_list = []
    val_acc_list = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Calculate statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Update train loss and accuracy once per epoch
        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        
        # Run validation loop once per epoch and update val accuracy
        val_acc = validate(model, val_loader, criterion, device)
        val_acc_list.append(val_acc)

        if epoch % 10 == 0:
            print('[Epoch %d] Train Loss: %.3f, Train Acc: %.3f, Val Acc: %.3f' %
              (epoch, train_loss, train_acc, val_acc))

    # Plot train and val accuracy once training is finished
    plt.plot(train_acc_list)
    plt.plot(val_acc_list)
    plt.title("Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(['Train', 'Validation'])
    plt.show()

    print('Finished Training')

In [391]:
train(mlp, train_loader, val_loader, criterion, optimizer, hparams['device'], hparams['num_epochs'])

[Epoch 0] Train Loss: 3.830, Train Acc: 3.460, Val Acc: 5.215
[Epoch 10] Train Loss: 2.764, Train Acc: 27.850, Val Acc: 23.583
[Epoch 20] Train Loss: 2.457, Train Acc: 33.919, Val Acc: 29.252
[Epoch 30] Train Loss: 2.284, Train Acc: 37.096, Val Acc: 31.293
[Epoch 40] Train Loss: 2.079, Train Acc: 42.541, Val Acc: 34.014
[Epoch 50] Train Loss: 1.971, Train Acc: 44.526, Val Acc: 32.426
[Epoch 60] Train Loss: 1.863, Train Acc: 47.476, Val Acc: 34.240
[Epoch 70] Train Loss: 1.778, Train Acc: 50.142, Val Acc: 34.694
[Epoch 80] Train Loss: 1.710, Train Acc: 50.596, Val Acc: 32.426
[Epoch 90] Train Loss: 1.655, Train Acc: 52.127, Val Acc: 34.921
[Epoch 100] Train Loss: 1.594, Train Acc: 52.864, Val Acc: 35.828
[Epoch 110] Train Loss: 1.559, Train Acc: 55.417, Val Acc: 34.467
[Epoch 120] Train Loss: 1.534, Train Acc: 55.417, Val Acc: 34.014


KeyboardInterrupt: ignored